# Imports

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import requests
import datetime

from time import sleep
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline


Bad key "text.kerning_factor" on line 4 in
/Users/davidsutton/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


# Get data

In [3]:
# Function to pull in posts (From Caroline/class)

def scrape_subreddit(subreddit):
    dataframes = []
    today = datetime.datetime.now()
    now = today.replace(hour=0, minute=0, second=0, microsecond=0)
    time = int(now.timestamp()) #get now in epoch date time format
    # could change to scrape more than 1000...
    for _ in range(10):
        res = requests.get(f'https://api.pushshift.io/reddit/search/submission/?subreddit={subreddit}&size=100&before={time}')
        df = pd.DataFrame(res.json()['data'])
        dataframes.append(df)
        time = res.json()['data'][-1]['created_utc']
        print('Still working on it...')
        sleep(5)
    all_posts = pd.concat(dataframes)
    all_posts.to_csv(f'subreddit_{now}.csv', index=False)
    return all_posts

In [4]:
# Get architecture data
architecture_df = scrape_subreddit('architecture')

Still working on it...
Still working on it...
Still working on it...
Still working on it...
Still working on it...
Still working on it...
Still working on it...
Still working on it...
Still working on it...
Still working on it...


In [5]:
architecture_df

,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,author_patreon_flair,author_premium,...,author_cakeday,media_metadata,author_flair_template_id,author_flair_text_color,gallery_data,is_gallery,author_flair_background_color,edited,poll_data,banned_by
0,[],False,Fine_Statistician_97,None,[],None,text,t2_7cupwbav,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],False,PigDino,None,[],None,text,t2_1miu3fn8,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],False,Noodlenomnom,None,[],None,text,t2_o808f,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],False,lonelyfatoldsickgirl,None,[],None,text,t2_135n8g,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[],False,11111111111000,None,[],None,text,t2_ycfd9,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,[],False,bud154,None,[],None,text,t2_1kjtto08,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,[],False,CreatorsArch,None,[],None,text,t2_3rjoe861,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,[],False,kill_bill3012,None,[],None,text,t2_3v0br6y0,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,[],False,Kazak_DogofSpace,None,[],None,text,t2_els01,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Get marine biology data

marine_df = scrape_subreddit('marinebiology')

Still working on it...
Still working on it...
Still working on it...
Still working on it...
Still working on it...
Still working on it...
Still working on it...
Still working on it...
Still working on it...
Still working on it...


In [7]:
# Concatenate
df = pd.concat([architecture_df, marine_df])[['title', 'subreddit']]

# Check for missing data

In [9]:
df.isnull().sum()

title        0
subreddit    0
dtype: int64

In [11]:
df.dtypes

title        object
subreddit    object
dtype: object

In [19]:
# Replace subreddit column with is_arch (1 for arch, 0 for marine biology)

df['is_arch'] = df['subreddit'].map(lambda x: 1 if x == 'architecture' else 0)

# Set up X and y, model and score

In [20]:
X = df['title']
y = df['is_arch']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [21]:
# Baseline accuracy

y_train.mean(), y_test.mean()

(0.49533333333333335, 0.514)

In [25]:
# Instantiate and fit Count Vectorizer

cv = CountVectorizer(min_df=3, max_features=2000)

cv.fit(X_train, y_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=2000, min_df=3,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [30]:
# Transform

X_train_cv = cv.transform(X_train)
X_test_cv = cv.transform(X_test)

In [31]:
# Instantiate and Fit

lr = LogisticRegressionCV(solver='liblinear')
lr.fit(X_train_cv, y_train)

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='auto', n_jobs=None,
                     penalty='l2', random_state=None, refit=True, scoring=None,
                     solver='liblinear', tol=0.0001, verbose=0)

In [32]:
# Score

lr.score(X_train_cv, y_train), lr.score(X_test_cv, y_test)

(0.9766666666666667, 0.888)

In [33]:
# Investigate coefs

coef_df = pd.DataFrame({
    'coefs': cv.get_feature_names(),
    'vals': lr.coef_[0]})

In [38]:
coef_df.sort_values('vals').head(10)

,coefs,vals
514,marine,-3.538680
313,fish,-2.861088
732,sea,-2.350175
748,shark,-2.165473
941,whale,-2.061453
327,found,-1.975851
588,octopus,-1.920220
700,research,-1.910851
585,ocean,-1.816282
269,eel,-1.741440


In [39]:
coef_df.sort_values('vals').tail(10)

,coefs,vals
658,practice,1.973410
180,city,2.031105
207,construction,2.133609
403,house,2.263072
234,design,2.455809
64,architect,2.478609
823,style,2.524145
65,architects,2.551391
139,building,2.848882
67,architecture,3.911863
